In [2]:
import pandas as pd
import numpy as np

import os

#os.listdir(".")
#credit = pd.read_csv("Credit_DataSet.csv")

# TODO: READ THE PROCESSED CSV INSTEAD (FOR EXAMPLE WITH CONVERTED CURRENCY)!

credit = pd.read_excel("./Credit/Credit_DataSet.xlsx", sheetname="DATA", skiprows=0)

# Quickly select numeric columns

to_keep = ['NBR_EMPLOYEES', 'COUNTRY_RISK_GROWTH_RATE',
       'COUNTRY_RISK_DIVIDEND_YIELD', 'COUNTRY_RISK_PAYOUT_RATIO',
       'VOLATILITY_30D', 'VOLATILITY_180D', 'PCT_CHG_1_YEAR', 'PCT_CHG_6_M',
       'DIVIDEND_YIELD', 'MARKETCAP', 'TOTAL_ASSETS', 'TOTAL_LIABILITIES',
       'CURRENT_ASSETS', 'EBIT', 'RETAINED_EARNINGS', 'SALES', 'SALES_GROWTH',
       'INTEREST_EXPENSES']

X = credit[ to_keep + ["DEFAULT_PROB"] ].dropna()

Y = X["DEFAULT_PROB"].values
X = X[to_keep].values

In [2]:
import pandas as pd
import numpy as np
# READ PREPROCESSED DATA FROM R FOR NN
X_df = pd.read_csv("some_X.csv", usecols=['NBR_EMPLOYEES', 'EMPL_GROWTH',
       'COUNTRY_RISK_GROWTH_RATE', 'COUNTRY_RISK_DIVIDEND_YIELD',
       'COUNTRY_RISK_PAYOUT_RATIO', 'VOLATILITY_30D', 'VOLATILITY_180D',
       'PCT_CHG_1_YEAR', 'PCT_CHG_6_M', 'DIVIDEND_YIELD', 'MARKETCAP',
       'TOTAL_ASSETS', 'TOTAL_LIABILITIES', 'CURRENT_ASSETS', 'EBIT',
       'RETAINED_EARNINGS', 'SALES', 'SALES_GROWTH', 'INTEREST_EXPENSES',
        'WHISTLE_BLOWER_POLICY', 'ETHICS_POLICY', 'BRIBERY_POLICY'])
Y_df = pd.read_csv("some_Y.csv", usecols=[1])

import torch
from torch import nn
from torch.autograd import Variable

In [52]:
def select_most_likely(mlp, loss_fn, x_row, y_row):
    candidates = [] # TODO: For every -1 in row in the specific columns, try 0 and 1
    
    bad_values = {'WHISTLE_BLOWER_POLICY': 0,
                            'ETHICS_POLICY': 0,
                            'BRIBERY_POLICY': 0}

    possible_values_dict = {'WHISTLE_BLOWER_POLICY':[[1,0], [0,1]],
                            'ETHICS_POLICY':[[1,0], [0,1]],
                            'BRIBERY_POLICY':[[1,0], [0,1]]}
    
    # Make a dictionary for the candidate values that we will expand later
    candidate_dict = {}
    for k, v in x_row.iteritems():
        candidate_dict[k] = []
        
        if (k in bad_values): # We have to encode for the neural network
            if (v == bad_values[k]):
                for i in possible_values_dict[k]:
                    # Directly encode the bad_value
                    candidate_dict[k].append(i)
            else:
                candidate_dict[k] = [possible_values_dict[k][int(v) - 1]] # HACK: IT'S BECAUSE THEY HAVE VALUES 1 AND 2
        else:
            # Just add the value
            candidate_dict[k].append([v])

    best_candidate = None
    lowest_loss = 100000000000

    # In the dictionary we have the different candidates
    for candidate in product(*candidate_dict.values()):
        input = Variable(torch.FloatTensor(np.array(list(chain(*list(candidate)))))).cuda()
        # Forward the candidate through the network        
        y_pred = mlp(input)
        loss = loss_fn(y_pred, y_row)
        if loss < lowest_loss:
            best_candidate = input
    return best_candidate
    
def get_E_step(mlp, loss_fn, X_df, Y_df):
    """ Return a torch CUDA variable of X"""
    df_list = []
    #for x_row, y_row in zip(X_df.iterrows(), Y_df.iterrows()):
    #    df_list.append(select_most_likely(mlp, loss_fn, x_row, y_row))
        
    for (k, j) in zip(X_df.iterrows(), Y_df.iterrows()):
        (_, x) = k
        (_, y) = j
        df_list.append(select_most_likely(model, loss_fn, x, Variable(torch.FloatTensor(y.values)).cuda()))

    return torch.stack(df_list)


In [ ]:
df_list = []
for (k, j) in zip(X_df.iterrows(), Y_df.iterrows()):
    (_, x) = k
    (_, y) = j
    df_list.append(select_most_likely(model, loss_fn, x, Variable(torch.FloatTensor(y.values)).cuda()))

df = pd.DataFrame(df_list)
#from itertools import * 

#select_most_likely(model, loss_fn, bla, 1)
#select_most_likely(model, loss_fn, bla, 1)

In [69]:
def get_M_step(mlp, loss_fn, X, Y, N_iter):
    learning_rate = 1e-5
    for t in range(N_iter):
        y_pred = model(X)
        loss = loss_fn(y_pred, Y)
        if (t % (N_iter / 2) == 0): print(loss)
        model.zero_grad()
        loss.backward()
        for param in model.parameters():
            param.data -= learning_rate * param.grad.data
    return mlp, loss

def EM_algorithm(X_df, Y_df, N_hidden=10, N_iter=10000):
    
    X = Variable(torch.FloatTensor(X_df.values)).cuda()
    Y = Variable(torch.FloatTensor(Y_df.values)).cuda() # Always stays the same

    # Make an initial model
    print("Making model")
    model = torch.nn.Sequential(
        torch.nn.Linear(X.shape[1] + 3, N_hidden),
        torch.nn.ReLU(),
        torch.nn.Linear(N_hidden, Y.shape[1]),
        torch.nn.Sigmoid() # If it doesn't work, look here
    )

    model = model.cuda()
    loss_fn = torch.nn.SoftMarginLoss(size_average=False)
    
    previous_loss = -50
    # Init
    for i in range(100):
        # Get the expected data
        print("Doing iteration {} E step".format(i))
        X = get_E_step(model, loss_fn, X_df, Y_df)
        # TODO: Check if the predicted data is the same still?
        # Train the model for some iterations
        print("Doing iteration {} M step".format(i))
        model, loss = get_M_step(model, loss_fn, X, Y, N_iter)
        if loss == previous_loss: 
            print("Loss not decreasing: exit")
            break
        previous_loss = loss
    return X, model
    

In [70]:
X_final, mlp = EM_algorithm(X_df, Y_df, N_hidden=100, N_iter=1000)

Making model
Doing iteration 0 E step
Doing iteration 0 M step
Variable containing:
 2694.2021
[torch.cuda.FloatTensor of size () (GPU 0)]

Variable containing:
 2694.2021
[torch.cuda.FloatTensor of size () (GPU 0)]

Doing iteration 1 E step
Doing iteration 1 M step
Variable containing:
 2694.2021
[torch.cuda.FloatTensor of size () (GPU 0)]

Variable containing:
 2694.2021
[torch.cuda.FloatTensor of size () (GPU 0)]

Loss not decreasing: exit


In [38]:
torch.stack(df_list)

Variable containing:
 -0.2657  -0.0431  -4.9499  ...    0.0000   1.0000   0.0000
 -0.3394  -0.0584   0.2982  ...    0.0000   1.0000   0.0000
 -0.3244  -0.0405  -4.9499  ...    0.0000   1.0000   0.0000
           ...               ⋱              ...            
 -0.3179  -0.0322   0.2982  ...    0.0000   1.0000   0.0000
 -0.3059  58.2679   0.2982  ...    0.0000   1.0000   0.0000
  0.2978  -0.0363   1.2324  ...    0.0000   1.0000   0.0000
[torch.cuda.FloatTensor of size (3933,25) (GPU 0)]